In [ ]:
!apt-get update

In [ ]:
!apt-get install openjdk-17-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz

In [ ]:
!tar -xvf spark-4.0.1-bin-hadoop3.tgz

In [ ]:
#!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/local/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home"
os.environ["SPARK_HOME"] = "/Users/mariabelakova/Documents/учеба/projects-Data-Engineer/spark-4.0.1-bin-hadoop3"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
df_iris = spark.read.csv('iris.csv', inferSchema=True, header=True)

In [ ]:
df_iris.describe().show()

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [ ]:
ohe = OneHotEncoder(inputCol='variety_num', outputCol='varietyOhe')

In [ ]:
model = ohe.fit(df_iris)

In [ ]:
df_iris_1 = model.transform(df_iris)

In [ ]:
df_iris_1.show()

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
df_iris_1.show()

In [ ]:
df_iris_1.columns

In [ ]:
assembler = VectorAssembler(inputCols=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], outputCol='Features')

In [ ]:
df_iris_1 = assembler.transform(df_iris_1)

In [ ]:
df_iris_1.show()

Или получим все колонки др способом через Pipeline



In [ ]:
from pyspark.ml import Pipeline

In [ ]:
pipeline = Pipeline(stages =
[
  OneHotEncoder(inputCol='variety_num', outputCol='varietyOhe'),
  VectorAssembler(inputCols=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'variety_num', 'varietyOhe'], outputCol='Features')
]
)

In [ ]:
pipelineTrained = pipeline.fit(df_iris)

In [ ]:
pipelineTrained.transform(df_iris).show()

In [ ]:
df_iris_2 = pipelineTrained.transform(df_iris)

In [ ]:
df_iris_2.show()

In [ ]:
df_iris.show()

In [ ]:
train, test = df_iris_1.randomSplit([0.8, 0.2], seed=12345)

In [ ]:
train.show()

## Создадим модель логистической регреcсии и обучим её.

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression(featuresCol = 'Features', labelCol = 'variety_num', maxIter=100)
lrModel = lr.fit(train)

In [ ]:
train_res = lrModel.transform(train)
test_res = lrModel.transform(test)

In [ ]:
train_res.show()

Воспользуемся MulticlassClassificationEvaluator для оценки качества на train и test множестве.

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
ev = MulticlassClassificationEvaluator(labelCol='variety_num', predictionCol='prediction', metricName='accuracy')

In [ ]:
accuracy_train = ev.evaluate(train_res)
accuracy_test = ev.evaluate(test_res)

print(f"Train accuracy: {accuracy_train:.3f}")
print(f"Test accuracy: {accuracy_test:.3f}")

## Выводы

- Модель хорошо обучена
- Очень высокая точность на тренировке показывает, что признаки (длина и ширина чашелистика и лепестка) позволяют эффективно различать классы ирисов.
- Нет признаков переобучения
- Если бы точность на тренировке была сильно выше, чем на тесте, это сигнал переобучения. Здесь тестовая точность даже 100%, значит модель хорошо обобщает.
- Iris dataset небольшой (150 записей, 3 класса) и линейно разделимый. Поэтому логистическая регрессия показывает почти идеальные результаты.
- Модель подходит для данной задачи
- Можно использовать такую модель для классификации ирисов на основе этих 4 признаков без дополнительной настройки.

# Итог
Модель логистической регрессии на признаках длины и ширины чашелистика и лепестка очень эффективно классифицирует ирисы. Переобучения нет, модель хорошо обобщает на тестовой выборке.